In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

import datetime as dt

from ipywidgets import interact
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sova.market_data import generate_market_data
from sova.asset import StockOption, OptionType
from sova.portfolio import Portfolio, Trade
from sova.simulation import HedgeSimulation

import importlib
import sova
importlib.reload(sova.asset);

In [30]:
a0 = 1e-5
n_obs = 1000
p_arr = np.array([.001])
q_arr = np.array([.002])

delta_threshold = .3

otm_multiplier = 1.05

# Saved generated data to pickle file in order to reproduce results.
# The same kind of data could be produced in the following way:
# md = generate_market_data(a0, p_arr, q_arr, n_obs=n_obs)
md = pd.read_pickle('simulation_data.pickle')
curr_mkt = md.iloc[0]
curr_mkt_time = curr_mkt.name.to_pydatetime()

opt = StockOption(amount=1,
                  expiry=md.index[-1].to_pydatetime(),
                  strike=otm_multiplier * md.asset_price[0],
                  sigma=.1)

portfolio = Portfolio([Trade(curr_mkt_time, curr_mkt.asset_price, opt)],
                      delta_threshold)
sim = HedgeSimulation(portfolio)
sim_df = sim.run_simulation(md)

fig = make_subplots(
    rows=3,
    cols=1,
    subplot_titles=['Asset price', 'Portfolio PV', 'Porfolio Delta'])

fig.append_trace(go.Scatter(x=md.index,
                            y=md.asset_price,
                            line_color='deepskyblue',
                            name='Asset price'),
                 row=1,
                 col=1)

fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[opt.strike, opt.strike],
                            line=dict(color='purple', width=4, dash='dash'),
                            name='Strike'),
                 row=1,
                 col=1)

fig.append_trace(go.Scatter(x=md.index,
                            y=sim_df.porfolio_pv,
                            line_color='limegreen',
                            name='Portfolio PV'),
                 row=2,
                 col=1)
fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[0, 0],
                            line=dict(color='firebrick', width=2),
                            name='Zero PV'),
                 row=2,
                 col=1)

fig.append_trace(go.Scatter(x=md.index,
                            y=sim_df.portfolio_delta,
                            line_color='mediumpurple',
                            name='Porfolio delta'),
                 row=3,
                 col=1)
fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[delta_threshold, delta_threshold],
                            line=dict(color='brown', width=2, dash='dash'),
                            name='Delta threshold'),
                 row=3,
                 col=1)
fig.append_trace(go.Scatter(x=[md.index[0], md.index[-1]],
                            y=[-delta_threshold, -delta_threshold],
                            line=dict(color='brown', width=2, dash='dash'),
                            name='Delta threshold'),
                 row=3,
                 col=1)

hedge_df = sim_df[np.abs(sim_df.hedge_asset_amount) > 0]
hedge_df['text'] = hedge_df.apply(
    lambda r:
    f'Hedged with ({r.hedge_deposit_amount} -> {r.hedge_asset_amount}) asset',
    axis=1)
fig.append_trace(go.Scatter(x=hedge_df.index,
                            y=hedge_df.portfolio_delta,
                            mode='markers',
                            marker=dict(size=15, color='magenta'),
                            text=hedge_df.text,
                            name='Delta hedge'),
                 row=3,
                 col=1)

fig.update_layout(height=1000, title_text="Delta Hedge Simulation Summary")
fig.show()